# Employee Turnover Analysis and Forecasting
This notebook analyzes employee turnover data and provides a forecast for future turnover rates using a SARIMA model.

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Dataset
df = pd.read_excel('employee_turnover.xlsx', parse_dates=['join_date', 'leave_date'])
df.head()

In [ ]:
# Preprocessing
df['turnover_date'] = df['leave_date'].dt.to_period('M').dt.to_timestamp()
turnover_monthly = (
    df[df['turnover'] == 1]
    .groupby('turnover_date')
    .size()
    .resample('M')
    .sum()
    .fillna(0)
)
turnover_monthly.head()

In [ ]:
# Time Series Forecasting (SARIMA Model)
model = SARIMAX(turnover_monthly, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results = model.fit()
print(results.summary())

In [ ]:
# Forecasting
forecast_steps = 12
forecast = results.get_forecast(steps=forecast_steps)
forecast_index = pd.date_range(turnover_monthly.index[-1] + pd.DateOffset(1), periods=forecast_steps, freq='M')
forecast_values = forecast.predicted_mean

In [ ]:
# Visualization
plt.figure(figsize=(10, 6))
plt.plot(turnover_monthly, label='Actual Turnover')
plt.plot(forecast_index, forecast_values, label='Forecasted Turnover', linestyle='--')
plt.title('Monthly Turnover Forecast')
plt.legend()
plt.show()